In [3]:
import json
import os
from collections import Counter
from dataclasses import asdict
from dataclasses import dataclass
from dataclasses import field
from pathlib import Path
from pprint import pprint
from typing import Self

import structlog
from lxml import etree
from lxml.etree import HTMLParser
from lxml.etree import _Element as Element
from structlog.stdlib import BoundLogger

# from data_adapter.standard_ebook_toc import Chapter
from utils.data_io import read_dict
from utils.data_io import read_xhtml
from utils.data_io import save_chunk
from utils.data_io import save_xhtml
from utils.logger_config import configure_logger


@dataclass
class Chapter:
    title: str
    number: str
    name: str
    href: str
    nest_level: int
    query: str
    url: str
    subchapters: list[Self]


def from_dict(chapter_data: dict) -> Chapter:
    """辞書データから `Chapter` オブジェクトを生成します。.

    Args:
        chapter_data (dict): 辞書データ

    Returns:
        Chapter: `Chapter` オブジェクト
    """
    subchapters = []
    if "subchapters" in chapter_data:
        for subchapter_data in chapter_data["subchapters"]:
            subchapters.append(from_dict(subchapter_data))

    return Chapter(
        title=chapter_data["title"],
        number=chapter_data["number"],
        name=chapter_data["name"],
        href=chapter_data["href"],
        nest_level=chapter_data["nest_level"],
        query=chapter_data["query"],
        url=chapter_data["url"],
        subchapters=subchapters,
    )


def load_chapters_from_json(json_path: str) -> list[Chapter]:
    """保存された JSON ファイルから `Chapter` オブジェクトのリストを読み込みます。.

    Args:
        json_path (str): JSON ファイルのパス

    Returns:
        List[Chapter]: `Chapter` オブジェクトのリスト
    """
    with open(json_path) as fp:
        data = json.load(fp)

    chapters = []
    for chapter_data in data:
        chapters.append(from_dict(chapter_data))

    return chapters


# def get_content_from_xhtml(xhtml: str) -> str:
#     parser = HTMLParser()
#     tree: Element = etree.fromstring(xhtml, parser)
#     content = tree.xpath("//body")[3]
#     return etree.tostring(content, pretty_print=True).decode("utf-8")


def show(element: Element | list[Element]) -> None:
    # This is for debugging.
    if len(element) == 0:
        print("No elements are found.")
    if isinstance(element, list):
        print(f"{len(element)} elements are found.")
        print("{:=^30}".format(" Show Data "))
        for _i, e in enumerate(element, 1):
            print(f"{_i:=^30}")
            show(e)
    if isinstance(element, Element):
        print(etree.tostring(element, pretty_print=True, encoding=str))
    else:
        print("No elements are found.")

In [4]:
BOOK_DIR = Path("/home/user/dev/kasi-x/akizora/tutorial_data/chaptered/nest_0/")

configure_logger()
logger = structlog.get_logger(__name__)
dirs = BOOK_DIR.glob("*")
for target_book in dirs:
    toc: list[Chapter] = load_chapters_from_json(target_book / "toc.json")

print(target_book)
target_files = []
for file in toc:
    match file.title:
        case "Titlepage":
            pass
        case "Imprint":
            pass
        case "Colophon":
            pass
        case "Uncopyright":
            pass
        case "halftitlepage":
            pass
        case _:
            target_files.append(file)
            # target_file_and_id[target_book + "/" + file.href.split("text/")[-1]] = (
            # file.href.split("text/")[-1].split("#")[-1].split(".xhtml")[0]
            # )
            # pprint(file)
            pprint(file.title)

            # read_xhtml(target_book + "/" + file.href.split("text/")[-1], logger))
result_dict = {}
for target_file in target_files:
    id = target_file.href.split("text/")[-1].split("#")[-1].split(".xhtml")[0]
    # print("=" * 40)
    # print(id)

    parser = HTMLParser()
    file_path = target_book / target_file.href.split("text/")[-1]
    xhtml = read_xhtml(file_path).encode("utf-8")
    tree = etree.fromstring(xhtml, parser)
    show(tree)
    # show(tree.xpath("//body")[0])
    etree.tostring(tree, pretty_print=True).decode("utf-8")

    # print(tree.xpath(f"//section[@id={id}"))
    print("=" * 40)
    # print(tree.xpath(f"//body/section[@id={id}]"))

    show(tree.xpath(f"//section[@id='{id}']/p"))
    for element in tree.xpath(f"//section[@id='{id}']/p"):
        for text in element.itertext():
            print(text.strip())
    print("+" * 40)
    result = []
    for element in tree.xpath(f"//section[@id='{id}']/p"):
        for text in element.itertext():
            text = text.strip()
            if len(text) == 0:
                result.append("\n")
            else:
                result.append(text)
    print(" ".join(result))
    result_dict[target_file.title] = " ".join(result)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/, se: https://standardebooks.org/vocab/1.0" xml:lang="en-US">
	<head>
		<title>CI: Locusta</title>
		<link href="../css/core.css" rel="stylesheet" type="text/css"/>
		<link href="../css/local.css" rel="stylesheet" type="text/css"/>
	</head>
	<body epub:type="bodymatter z3998:fiction">
		<section id="chapter-101" epub:type="chapter">
			<hgroup>
				<h2 epub:type="ordinal z3998:roman">CI</h2>
				<p epub:type="title">Locusta</p>
			</hgroup>
			<p>Valentine was alone; two other clocks, slower than that of Saint-Philippe-du-Roule, struck the hour of midnight from different directions, and excepting the rumbling of a few carriages all was silent. Then Valentine’s attention was engrossed by the clock in her room, which marked the seconds. She began counting them, remarking that they were much slower than the beatings of her heart; and stil

In [58]:
def main():
    configure_logger()
    logger = structlog.get_logger(__name__)
    book_paths = read_dict(BOOK_DIR / "easy_readable_books.json", logger)
    target_book = book_paths[0]
    toc: list[Chapter] = load_chapters_from_json(target_book + "/toc.json")

    print(target_book)
    target_files = []
    for file in toc:
        match file.title:
            case "Titlepage":
                pass
            case "Imprint":
                pass
            case "Colophon":
                pass
            case "Uncopyright":
                pass
            case "halftitlepage":
                pass
            case _:
                target_files.append(file)
                # target_file_and_id[target_book + "/" + file.href.split("text/")[-1]] = (
                # file.href.split("text/")[-1].split("#")[-1].split(".xhtml")[0]
                # )
                # pprint(file)
                pprint(file.title)

                # read_xhtml(target_book + "/" + file.href.split("text/")[-1], logger))
    result_dict = {}
    for target_file in target_files:
        id = target_file.href.split("text/")[-1].split("#")[-1].split(".xhtml")[0]
        # print("=" * 40)
        # print(id)

        parser = HTMLParser()
        file_path = target_book + "/" + target_file.href.split("text/")[-1]
        xhtml = read_xhtml(file_path).encode("utf-8")
        tree = etree.fromstring(xhtml, parser)
        show(tree)
        # show(tree.xpath("//body")[0])
        etree.tostring(tree, pretty_print=True).decode("utf-8")

        # print(tree.xpath(f"//section[@id={id}"))
        print("=" * 40)
        # print(tree.xpath(f"//body/section[@id={id}]"))

        show(tree.xpath(f"//section[@id='{id}']/p"))
        for element in tree.xpath(f"//section[@id='{id}']/p"):
            for text in element.itertext():
                print(text.strip())
        print("+" * 40)
        result = []
        for element in tree.xpath(f"//section[@id='{id}']/p"):
            for text in element.itertext():
                text = text.strip()
                if len(text) == 0:
                    result.append("\n")
                else:
                    result.append(text)
        print(" ".join(result))
        result_dict[target_file.title] = " ".join(result)
    return result_dict



In [59]:
a = main()

2024-04-19 06:43.19 [info     ] Reading data.                  [__main__] path=PosixPath('/home/user/dev/kasi-x/akizora/books/easy_readable_books.json')
2024-04-19 06:43.19 [info     ] Reading data.                  [__main__] path=PosixPath('/home/user/dev/kasi-x/akizora/books/easy_readable_books.json')
2024-04-19 06:43.19 [info     ] Reading data.                  [__main__] path=PosixPath('/home/user/dev/kasi-x/akizora/books/easy_readable_books.json')
2024-04-19 06:43.19 [info     ] Reading data.                  [__main__] path=PosixPath('/home/user/dev/kasi-x/akizora/books/easy_readable_books.json')
2024-04-19 06:43.19 [info     ] Reading data.                  [__main__] path=PosixPath('/home/user/dev/kasi-x/akizora/books/easy_readable_books.json')
2024-04-19 06:43.19 [info     ] Reading data.                  [__main__] path=PosixPath('/home/user/dev/kasi-x/akizora/books/easy_readable_books.json')
2024-04-19 06:43.19 [info     ] Reading data.                  [__main__] path=Pos

In [60]:
a

{'Dedication': 'Dedicated to two distinguished members of the O.F.D. \n Carlotta \n and \n Peter \n',
 'The Mystery of the Blue Train': ''}

In [61]:
pprint(a)

{'Dedication': 'Dedicated to two distinguished members of the O.F.D. \n'
               ' Carlotta \n'
               ' and \n'
               ' Peter \n',
 'The Mystery of the Blue Train': ''}


In [11]:
target_file = '/home/user/dev/kasi-x/akizora/books/agatha-christie_the-mystery-of-the-blue-train/dedication.xhtml'

In [16]:
xhtml = read_xhtml(target_file).encode("utf-8")

In [17]:
parser = HTMLParser()
tree = etree.fromstring(xhtml, parser)
content = tree.xpath("//body")[0]
a = etree.tostring(content, pretty_print=True).decode("utf-8")
show(tree.xpath("//section[@id='dedication'][1]/p"))


In [26]:
def show(element: Element | list[Element]) -> None:
    # This is for debugging.
    if len(element) == 0:
        print("No elements are found.")
    if isinstance(element, list):
        print(f"{len(element)} elements are found.")
        print("{:=^30}".format(" Show Data "))
        for _i, e in enumerate(element, 1):
            print(f"{_i:=^30}")
            show(e)
    if isinstance(element, Element):
        print(etree.tostring(element, pretty_print=True, encoding=str))
    else:
        print("No elements are found.")

In [27]:
show(tree)

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/, se: https://standardebooks.org/vocab/1.0" xml:lang="en-GB">
	<head>
		<title>Dedication</title>
		<link href="../css/core.css" rel="stylesheet" type="text/css"/>
		<link href="../css/local.css" rel="stylesheet" type="text/css"/>
	</head>
	<body epub:type="frontmatter z3998:fiction">
		<section id="dedication" epub:type="dedication">
			<p><span>Dedicated to</span><br/>
			two distinguished members<br/>
			of the <abbr class="eoc" epub:type="z3998:initialism">O.F.D.</abbr></p>
			<p>
				<b>Carlotta</b>
				<i>and</i>
				<b>Peter</b>
			</p>
		</section>
	</body>
</html>



In [ ]:
//section[@id='dedication'][1]/@epub:type

In [29]:
show(tree.xpath("//section[@id='dedication'][1]/p"))

2 elements are found.
========= Show Data ==========
==============1===============
<p><span>Dedicated to</span><br/>
			two distinguished members<br/>
			of the <abbr class="eoc" epub:type="z3998:initialism">O.F.D.</abbr></p>
			

==============2===============
<p>
				<b>Carlotta</b>
				<i>and</i>
				<b>Peter</b>
			</p>
		

No elements are found.


In [15]:
file

{'title': 'Titlepage',
 'number': '',
 'name': 'Titlepage',
 'href': 'text/titlepage.xhtml',
 'nest_level': 0,
 'query': "//html/body/nav[@id='toc'][1]/ol/li[1]",
 'url': '',
 'subchapters': []}

In [16]:
file["href"].split("/")[-1]

'titlepage.xhtml'

In [20]:
a = 'text/chapter-35.xhtml'

In [21]:
a.split("/")[-1]

'chapter-35.xhtml'